In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request, json , time, os, difflib, itertools
import re
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime
try:
    import httplib
except:
    import http.client as httplib

In [2]:
def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        return True
    except:
        conn.close()
        return False

In [3]:
def get_historic_price(query_url,json_path,csv_path):
    
    stock_id=query_url.split("&period")[0].split("symbol=")[1]

    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists")
        return
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)

    try:
        with urllib.request.urlopen(query_url) as url:
            parsed = json.loads(url.read().decode())
    
    except:
        print("|||  Historical data of "+stock_id+" doesn't exist")
        return
    
    else:
        if os.path.exists(json_path+stock_id+'.json') and os.stat(json_path+stock_id+'.json').st_size != 0:
            os.remove(json_path+stock_id+'.json')
        
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)
        
        try:
            Date = []
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))

            Adjusted_Close = parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']
            #Marker_Cap = parsed['chart']['result'][0]['indicators']['marketCap'][0]['marketCap']

            df = pd.DataFrame(list(zip(Date,Adjusted_Close)),columns=['Date','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
        
        except:
            print(">>>  Historical data of "+stock_id+" could not be saved")
        
        return

In [4]:
json_path = "historic_data"+os.sep+"json"+os.sep
csv_path = "historic_data"+os.sep+"csv"+os.sep

In [5]:
if not os.path.isdir(json_path):
    os.makedirs(json_path)
if not os.path.isdir(csv_path):
    os.makedirs(csv_path)

In [6]:
ticker_file_path = "Yahoo Ticker Symbols - September 2017.xlsx"
temp_df = pd.read_excel(ticker_file_path)
print("Total stocks:",len(temp_df))
temp_df.head(10)

Total stocks: 106331


,Yahoo Stock Tickers,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,http://investexcel.net,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticker,Name,Exchange,Category Name,Country,NaN,NaN,NaN
3,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA,NaN,NaN,Samir Khan
4,AAPL,Apple Inc.,NMS,Electronic Equipment,USA,NaN,NaN,simulationconsultant@gmail.com
5,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA,NaN,NaN,NaN
6,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA,NaN,NaN,This ticker symbol list was downloaded from
7,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA,NaN,NaN,http://investexcel.net/all-yahoo-finance-stock...
8,GOOG,Alphabet Inc.,NMS,Internet Information Providers,USA,NaN,NaN,and was updated on 2nd September 2017
9,MO,"Altria Group, Inc.",NYQ,Cigarettes,USA,NaN,NaN,NaN


In [7]:
temp_df = temp_df.drop(temp_df.columns[[5, 6, 7]], axis=1)
headers = temp_df.iloc[2]
df  = pd.DataFrame(temp_df.values[3:], columns=headers)
print("Total stocks:",len(df))
df.head(10)

Total stocks: 106328


2,Ticker,Name,Exchange,Category Name,Country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA
5,GOOG,Alphabet Inc.,NMS,Internet Information Providers,USA
6,MO,"Altria Group, Inc.",NYQ,Cigarettes,USA
7,DAL,"Delta Air Lines, Inc.",NYQ,Major Airlines,USA
8,AA,Alcoa Corporation,NYQ,Aluminum,USA
9,AXP,American Express Company,NYQ,Credit Services,USA


In [8]:
query_urls=[]
for ticker in df['Ticker']:
    query_urls.append("https://query1.finance.yahoo.com/v8/finance/quoteSummary/"+ticker+"?symbol="+ticker+"&period1=0&period2=9999999999&interval=1d&includePrePost=true&events=div%2Csplit")

In [ ]:
with Pool(processes=10) as pool:
    pool.starmap(get_historic_price, zip(query_urls, itertools.repeat(json_path), itertools.repeat(csv_path)))
print("<|>  Historical data of all stocks saved")